#### 0. Autora
<br>
Carolina Saavedra - carolinasaavedra01@gmail.com
<br>

#### 1. Título
<br>
01_Resumen_CV's
<br>

#### 2. Propósito
<br>
Este script en Python tiene como propósito automatizar la extracción de información clave desde archivos PDF que contienen currículums (CVs) de postulantes, estructurar esa información y exportarla en un archivo `.xlsx` para análisis, clasificación o gestión posterior.

- Identificar y extraer automáticamente secciones importantes como:
  - Nombre
  - Correo electrónico
  - Número de teléfono
  - Perfil de LinkedIn
  - Formación académica
  - Experiencia laboral
  - Idiomas
  - Actividades extracurriculares
- Consolidar todos los datos en un único archivo de Excel (`Resumen_Postulantes.xlsx`).

#### 3. ¿Qué hace el script?

1. **Recorre todos los archivos PDF** de una carpeta especificada.
2. **Extrae el texto completo** de cada archivo.
3. **Busca patrones específicos** con expresiones regulares para detectar:
   - Correos electrónicos (`@`)
   - Números de teléfono (con o sin +51)
   - URLs de LinkedIn
   - Secciones como “Educación”, “Experiencia”, “Idiomas” o “Voluntariados”, incluso con diferentes variantes de nombre.
4. **Crea un diccionario por CV** con todos los datos estructurados.
5. **Guarda toda la información en un DataFrame** de pandas.
6. **Exporta un archivo de Excel** con todos los datos extraídos.

#### 4. Paquetes a instalar
Instala las siguientes librerías antes de ejecutar el script:

```bash
pip install pymupdf pandas openpyxl
```

#### 5. Instrucciones
En el directorio `pdf_folder_path` coloca la dirección donde se encuentren todos los archivos de CV's en formato PDF. 


#### 6. Código

#### 6.1. Importar paquetes

In [1]:
import fitz 
import os
import re
import pandas as pd

C:\Users\CAROLINA\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


#### 6.2. Directorio

In [2]:
pdf_folder_path = 'C:\\Users\\CAROLINA\\Documents\\02_GitHub\\CV_Structure' # En esta ruta colocarás todos los CV's 

#### 6.3. Funciones de automatización

In [3]:
# Función: Extraer texto de un PDF
def extract_text_from_pdf(pdf_path, max_pages=300):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc[:max_pages]:
            text += page.get_text("text")
        doc.close()
        return text
    except Exception as e:
        print(f"❌ Error al abrir {pdf_path}: {e}")
        return ""

# Función: Extraer sección del texto
def extract_section(text, section_keywords):
    lines = text.split("\n")
    section_text = []
    collecting = False
    keywords = [kw.lower() for kw in section_keywords]

    for line in lines:
        line_lower = line.strip().lower()
        if any(kw in line_lower for kw in keywords):
            collecting = True
            continue
        elif collecting and any(kw in line_lower for kw in [
            "educación", "educacion", "estudios", 
            "experiencia", "idiomas", "lenguas", "lenguajes", 
            "actividades", "voluntariado", "otros"
        ]):
            break
        elif collecting:
            section_text.append(line.strip())

    return " ".join(section_text).strip()

# Función: Extraer campos clave. Puedes añadir otras palabras clave
def extract_fields_from_text(text, filename):
    lines = text.strip().split("\n")
    lines = [line.strip() for line in lines if line.strip()]
    full_text = "\n".join(lines)

    candidate_name = lines[0] if lines else ""

    email_match = re.search(r'[\w\.-]+@[\w\.-]+\.\w+', full_text)
    phone_match = re.search(r'(\+?\d[\d\s\-]{7,20})', full_text)
    linkedin_match = re.search(r'(linkedin\.com/in/[^\s]+)', full_text)

    education_text = extract_section(full_text, [
        "educación", "educacion", "estudios", "formación académica", "formacion academica"
    ])
    experience_text = extract_section(full_text, [
        "experiencia", "experiencia laboral", "experiencia profesional", "trayectoria", "historial laboral", "EXPERIENCIA"
    ])
    language_text = extract_section(full_text, [
        "idiomas", "lenguas", "lenguajes", "competencias lingüísticas", "habilidades idiomáticas"
    ])
    extras_text = extract_section(full_text, [
        "actividades extracurriculares", "voluntariado", "voluntariados", "otros datos", "información adicional"
    ])

    fields = {
        "Nombre": candidate_name or "",
        "Correo": email_match.group() if email_match else "",
        "Teléfono": phone_match.group() if phone_match else "",
        "LinkedIn": linkedin_match.group() if linkedin_match else "",
        "Educación": education_text or "",
        "Experiencia": experience_text or "",
        "Idiomas": language_text or "",
        "Extras": extras_text or "",
        "Archivo": filename
    }
    return fields

# Función principal: Procesar todos los PDFs y generar CSV
def process_pdfs_to_dataframe(folder_path, max_pages=300):
    data = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            text = extract_text_from_pdf(pdf_path, max_pages)
            fields = extract_fields_from_text(text, filename)
            data.append(fields)

    df = pd.DataFrame(data)
    expected_cols = ["Nombre", "Correo", "Teléfono", "LinkedIn", "Educación", "Experiencia", "Idiomas", "Extras", "Archivo"]
    for col in expected_cols:
        if col not in df.columns:
            df[col] = ""
    df = df[expected_cols]
    return df

# Ejecutar. Puedes cambiar el nombre del archivo si deseas.
if __name__ == "__main__":
    df_resultado = process_pdfs_to_dataframe(pdf_folder_path)
    df_resultado.to_excel(os.path.join(pdf_folder_path, "Resumen_Postulantes.xlsx"), index=False, engine='openpyxl')
    print("✅ Excel generado con éxito: Resumen_Postulantes.xlsx")

✅ Excel generado con éxito: Resumen_Postulantes.xlsx
